### Templates

like our last view **list** we don't have to use html inside our python function, ( infact we shouldn't use html inside our python file at all ). So we have templates ( which are plain html files with some of python like constructs. )

to use templates we will have to put html files on some location, we will use templates folder inside app directory.

    $ mkdir -p property/templates

In [2]:
%ll ../property

total 28
-rw-r--r-- 1 idwaker  336 Oct 13 10:34 admin.py
-rw-r--r-- 1 idwaker    0 Oct 12 14:37 __init__.py
drwxr-xr-x 3 idwaker 4096 Oct 13 10:28 migrations/
-rw-r--r-- 1 idwaker  662 Oct 13 10:44 models.py
drwxr-xr-x 2 idwaker 4096 Oct 15 09:45 __pycache__/
drwxr-xr-x 2 idwaker 4096 Oct 15 08:44 templates/
-rw-r--r-- 1 idwaker   60 Oct 12 14:37 tests.py
-rw-r--r-- 1 idwaker 1887 Oct 15 10:15 views.py


In [5]:
%ll ../property/templates

total 0
-rw-r--r-- 1 idwaker 0 Oct 15 10:35 list.html


we have **list.html** which we will use with **list** view on views.py instead of showing html contents from python function itself.

change our **list** view in *property/views.py* file as
```python
def list(request):
    """
    This view list all houses
    """

    # get all house objects/rows stored in database
    houses = House.objects.all()

    # get only those added by above add view
    selected = House.objects.filter(title='This is added and also duplicated')
    return render(request, 'list.html', {
        'all_houses': houses,
        'selected_houses': selected
    })
```

Here
>render will load *list.html* file from templates directory and will pass **houses** variable as **all_houses** variable and **selected** variable as **selected_houses** variable

we need to import *render* at top of the file
```python
from django.shortcuts import render
```

*property/templates/list.html* looks like
```django
<!DOCTYPE html>
<html>
    <head>
        <title>List of Houses</title>
    </head>
    <body>
        <div>
            <h3>All Houses</h3>
            <ul>
            {% for house in all_houses %}
                <li>{{ house.title }}</li>
            {% endfor %}
            </ul>

            <h3>Selected Houses</h3>
            <ul>
            {% for house in selected_houses %}
                <li>{{ house.title }}</li>
            {% endfor %}
            </ul>
        </div>
    </body>
</html>
```

we have remove all logic that is related to listing houses from **list** view and moved to **list** template using html content.

In above template code. we have use two kinds of tags,

1. that is inside angle brackets < and >, starts with **tag-name** and ends with **/tag-name**. These are plain html tags see [Html Tutorial](http://www.w3schools.com/html/default.asp)
2. Other that is inside {% and %} and {{ and }}. These are django specific tags and shouldn't be uses outside django. ( But there are similar templates used in other frameworks/languages as well, like Jinja2 in python and Twig in Php, all inspired from Django and works very much similar. ) [See More](https://docs.djangoproject.com/en/1.8/topics/templates/)
    
    - {% %} are control statements like **for** loop, or **if/else** block, each control statement must end with **end** statement like **endfor** **endif**, **endblock** for **block** statement etc.
    - {{ }} are just used for printing variables or showing values of variables.

>Like above we loop through each of **all_houses** variable and print **title** for each house, same as we did previosuly but inside function, this is much cleaner and easy to undestand

Reload the page and you will not see any changes, because both results are equivalent.

We have updated our **list** view, let's update our **add** view too. We will create a form so we will be able to create new House, instead creating same House each time we reload page.

*property/views.py*
```python
def add(request):
    """
    This view shows how we can create a new instance and save to database

    Every view must have request argument, which is passed by django
    """
    done = ''
    if request.POST:
        house = House(
            title=request.POST.get('title'),
            description=request.POST.get('description'),
            price=request.POST.get('price'),
            address=request.POST.get('address'),
            bedrooms=request.POST.get('bedrooms'),
            bathrooms=request.POST.get('bathrooms'),
            kitchens=request.POST.get('kitchens')
        )
        house.save()
        done = 'Your form is submitted and saved.'

    return render(request, 'form.html', {'done': done})
```

>In above code if form is submitted by clicking Save button, a new house object is creating using those submitted from the **form**, all the submitted data goes to *request.POST* dictionary and we can access using method above, Remember: item we are accessing must be name attribute on html form.

>In either case, form submitted or not we will get the same form, however if form is submitted our **done** variable will have message, which will be displayed later on.


*property/form.html*
```django
<!DOCTYPE html>
<html>
    <head>
        <title>Add New Houses</title>
    </head>
    <body>
        <div>
            <h3>Add House: {{ done }}</h3>
            <form name="add" action="" method="post">
                {% csrf_token %}
                <div>
                    <label>Title</label>
                    <input type="text" name="title" value="" />
                </div>
                <div>
                    <label>Description</label>
                    <input type="text" name="description" value="" />
                </div>
                <div>
                    <label>Price</label>
                    <input type="text" name="price" value="" />
                </div>
                <div>
                    <label>Address</label>
                    <textarea name="address"></textarea>
                </div>
                <div>
                    <label>Bedrooms</label>
                    <input type="text" name="bedrooms" value="" />
                </div>
                <div>
                    <label>Bathrooms</label>
                    <input type="text" name="bathrooms" value="" />
                </div>
                <div>
                    <label>Kitchens</label>
                    <input type="text" name="kitchens" value="" />
                </div>
                <div>
                    <input type="submit" name="submit" value="Save" />
                </div>
            </form>
        </div>
    </body>
</html>
```

>csrf_token creates a hidden input field with csrf token generated by django, csrf token is automatically verified by django while submitting any form. [Learn more about CSRF](https://en.wikipedia.org/wiki/Cross-site_request_forgery)

we can verify from screenshots

![view-add2](./images/view-add2.png)

![view-add3](./images/view-add3.png)

![view-list5](./images/view-list5.png)

Adding data was easy, but we need to consider many things while we are creating such forms.

- What happens when user submitted blank form ?
- What happens when user submitted wrong type on some field, for example add 'abc' to number of bedrooms ? and many more..

so to make sure user's are not allowed to do anything other than what we want them to do, we validate and sanitize forms after they are submitted and we only save to database after we are sure they are safe to save.

Here is screenshot of what happens when we try to submit empty form.

![view-add-error1](./images/view-add-error1.png)

and it is not nice to show this kind of error message to users.

django gives a easy way to validate and show error messages for such forms, let's use that

first we create *forms.py* module inside property app.

*property/forms.py*
```python
from django import forms


class HouseForm(forms.Form):
    title = forms.CharField(max_length=120)
    description = forms.CharField(widget=forms.Textarea)
    price = forms.DecimalField(max_digits=9, decimal_places=2)
    address = forms.CharField(widget=forms.Textarea)
    bedrooms = forms.IntegerField()
    bathrooms = forms.IntegerField()
    kitchens = forms.IntegerField()
```

> a form definition is same as model definition, but it doesnot change or anyway relate to database.

[Learn More about forms](https://docs.djangoproject.com/en/1.8/topics/forms/)

now change **add** view in *property/views.py*
```python
from .forms import HouseForm


def add(request):
    """
    This view shows how we can create a new instance and save to database

    Every view must have request argument, which is passed by django
    """
    form = HouseForm()
    done = ''
    if request.POST:
        form = HouseForm(request.POST)
        if form.is_valid():
            house = House(
                title=form.cleaned_data['title'],
                description=form.cleaned_data['description'],
                price=form.cleaned_data['price'],
                address=form.cleaned_data['address'],
                bedrooms=form.cleaned_data['bedrooms'],
                bathrooms=form.cleaned_data['bathrooms'],
                kitchens=form.cleaned_data['kitchens']
            )
            house.save()
            done = 'Your form is submitted and saved.'

    return render(request, 'form.html', {'done': done, 'form': form})
```

> only difference is that we are creating HouseForm object, if form is submitted then we pass submitted data to HouseForm. Check is form is valid ( rules are derived from HouseForm attributes, such as every field is required unless defined blank=True, type must be matched and so on

we make changes on template as well *property/templates/form.html'
```django
<!DOCTYPE html>
<html>
    <head>
        <title>Add New Houses</title>
    </head>
    <body>
        <div>
            <h3>Add House: {{ done }}</h3>
            <form name="add" action="" method="post">
                {% csrf_token %}
                {{ form.as_p }}
                <div>
                    <input type="submit" name="submit" value="Save" />
                </div>
            </form>
        </div>
    </body>
</html>
```

which got a lot shorter, because form object outputs the necessary html by itself. Reload the page, there is not much difference instead of validation while we try to submit the page.

here is screenshot of submitting empty form ( where there was error in screenshot above)
![view-add4](./images/view-add4.png)